In [108]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import itertools
# X, Y, Z - współrzędne geodezujne, nie matematyczne...
%matplotlib inline

In [111]:
def delta_vector(A, B):
    """zwraca wektor delt między dwoma punktami A do B. A,B: np.array()"""
    if len(A) != len(B):
        print("A and B dimensions are not the same (they should be)")
        return
    delts = []
    for i in range(len(A)):
        delts.append(B[i]-A[i])
    return np.array(delts)

In [112]:
def geo2xyz(fi, lam, h, a=6378137, e2=0.00669437999013):
    # dorobic domusllne a i e2 dla elipsoidy GRS80
    """funkcja zamienia współrzędne geodezyjne na kartezjańskie fi, lam podajemy w radianachpython """
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    x = (N + h) * np.cos(fi) * np.cos(lam)
    y = (N + h) * np.cos(fi) * np.sin(lam)
    z = (N * (1 - e2) + h) * np.sin(fi)
    return np.array([x, y, z])

In [113]:
def xyz2neu(fi, lam, A, B):
    # def xyz2neu(A:punktodniesienia/startowy, B:koniecwektora):
    """funkcja zamienia wsp kartezjańskie na topocentryczne neu
    A, B reprezentują punkty, A to początek, B to koniec wektora
    A, B są typu np.array i mają 3 współrzędne: x, y, z
    fi, lam to współrzędne punktu A potrzebne do macierzy obrotu"""
    # x, y, z -> north, east, up
    # fi, lambda, lotniska
    # wektor AB (od lotniska do samolotu)
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    rotation_matrix = np.array([
        [-1*np.sin(fi)*np.cos(lam), -1*np.sin(lam), np.cos(fi)*np.cos(lam)],
        [-1*np.sin(fi)*np.sin(lam), np.cos(lam), np.cos(fi)*np.sin(lam)],
        [np.cos(fi), 0, np.sin(fi)]
    ])
    vector = delta_vector(A, B)
    return rotation_matrix.transpose() @ vector

In [116]:
delta_vector([-2.6,-2,12], [1,2,3])

array([ 3.6,  4. , -9. ])

In [ ]:
geo2xyz(52, 21, 150)

In [ ]:
xyz2neu(52,21,[3673688.28615385, 1410196.80898536, 5002921.54709572], [3673788.28615385, 1410196.80898536, 5002921.54709572])

In [ ]:
geo2xyz(0,0,0)

In [ ]:
xyz2neu(0,0,[6378137.,       0.,       0.], [6378137.,       20,       40])

In [34]:
orig_df = pd.read_csv('radar.csv', sep=';')

In [36]:
orig_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917 entries, 0 to 916
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   917 non-null    float64
 1   Longitude  917 non-null    float64
 2   Meters     917 non-null    int64  
 3   kmh        917 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 28.8 KB


In [38]:
orig_df.describe(include='all')

,Latitude,Longitude,Meters,kmh
count,917.000000,917.000000,917.000000,917.000000
mean,53.666378,71.506575,10640.327154,815.562704
std,7.365936,33.348366,3131.306929,109.098678
min,36.842500,20.828500,115.000000,232.000000
25%,52.304000,41.148000,11575.000000,798.000000
50%,56.763000,72.378400,12184.000000,848.000000
75%,58.871500,100.163300,12192.000000,872.000000
max,60.139800,126.437300,12192.000000,914.000000


In [104]:
main_df = orig_df
main_df['xyz'] = pd.Series(map(geo2xyz, main_df['Latitude'], main_df['Longitude'], main_df['Meters']))

In [105]:
pd.set_option('display.max_colwidth', None)
main_df

,Latitude,Longitude,Meters,kmh,xyz
0,37.4806,126.4373,115,327,"[-3009986.9279177887, 4077087.335196061, 3859925.648851618]"
1,37.4926,126.4277,298,333,"[-3008908.6770793367, 4077056.266483708, 3861093.862974308]"
2,37.5041,126.4187,413,352,"[-3007861.0886884076, 4076977.0155891557, 3862176.543878538]"
3,37.5174,126.4078,474,391,"[-3006580.627028365, 4076864.661696517, 3863384.693334532]"
4,37.5421,126.3868,603,447,"[-3004156.090917171, 4076704.2115607024, 3865637.4877571473]"
...,...,...,...,...,...
912,52.2403,20.9018,693,232,"[3656694.4515809016, 1396487.460066755, 5019768.603065541]"
913,52.2294,20.9113,523,232,"[3657261.3345508967, 1397398.8339414, 5018891.353449281]"
914,52.1966,20.9400,358,246,"[3659160.510699166, 1400225.388276155, 5016524.526847928]"
915,52.1872,20.9483,282,253,"[3659685.91079667, 1401034.2542547686, 5015823.264119978]"


In [126]:
main_df['neu'] = pd.Series(map(xyz2neu, 
                               list(itertools.repeat(52.1786, 917)), 
                               list(itertools.repeat(20.9559, 917)), 
                               list(itertools.repeat([3660166.5031345594, 1401774.9245151733, 5015181.999035442], 
                                                     917)), main_df['xyz']
                              )
                          )

In [127]:
main_df

,Latitude,Longitude,Meters,kmh,xyz,neu
0,37.4806,126.4373,115,327,"[-3009986.9279177887, 4077087.335196061, 3859925.648851618]","[3456194.312054833, 4883932.042864793, -4145454.040411133]"
1,37.4926,126.4277,298,333,"[-3008908.6770793367, 4077056.266483708, 3861093.862974308]","[3456124.0398608404, 4883517.393530443, -4143920.5991438404]"
2,37.5041,126.4187,413,352,"[-3007861.0886884076, 4076977.0155891557, 3862176.543878538]","[3456037.5497620427, 4883068.715379161, -4142482.8483097763]"
3,37.5174,126.4078,474,391,"[-3006580.627028365, 4076864.661696517, 3863384.693334532]","[3455865.564098778, 4882505.836839043, -4140819.8933348134]"
4,37.5421,126.3868,603,447,"[-3004156.090917171, 4076704.2115607024, 3865637.4877571473]","[3455503.787353974, 4881488.865935644, -4137687.1485192087]"
...,...,...,...,...,...,...
912,52.2403,20.9018,693,232,"[3656694.4515809016, 1396487.460066755, 5019768.603065541]","[6867.564306236855, -3695.9535824309232, 475.2330267139282]"
913,52.2294,20.9113,523,232,"[3657261.3345508967, 1397398.8339414, 5018891.353449281]","[5653.978218917138, -3047.606921202584, 306.76657360920626]"
914,52.1966,20.9400,358,246,"[3659160.510699166, 1400225.388276155, 5016524.526847928]","[2003.1080726386724, -1087.251220554971, 144.59286333778505]"
915,52.1872,20.9483,282,253,"[3659685.91079667, 1401034.2542547686, 5015823.264119978]","[956.9986575715795, -519.7958519587401, 68.90704044991122]"
